In [1]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-65-2-83-175.ap-south-1.compute.amazonaws.com:5000/")

In [2]:
# Set or create an experiment
mlflow.set_experiment("Exp - 9 ML Algos with HP Tuning")

2024/10/14 16:53:40 INFO mlflow.tracking.fluent: Experiment with name 'Exp - 9 ML Algos with HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://youtube-comments-sentiment/336310026076603578', creation_time=1728905039812, experiment_id='336310026076603578', last_update_time=1728905039812, lifecycle_stage='active', name='Exp - 9 ML Algos with HP Tuning', tags={}>

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna

In [4]:
df = pd.read_csv('Preprocessed_Reddit.csv').dropna()
df.shape

(36662, 2)

In [5]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for KNN

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for KNN
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 30)  # Tuning the number of neighbors
    p = trial.suggest_categorical('p', [1, 2])  # Tuning the distance metric (1 for Manhattan, 2 for Euclidean)

    # KNeighborsClassifier setup
    model = KNeighborsClassifier(n_neighbors=n_neighbors, p=p)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for KNN, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_knn, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], p=best_params['p'])

    # Log the best model with MLflow, passing the algo_name as "KNN"
    log_mlflow("KNN", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for KNN
run_optuna_experiment()

[I 2024-10-14 16:54:49,345] A new study created in memory with name: no-name-08bad4bc-6430-4e3d-9516-86510644c6b0
[I 2024-10-14 16:54:58,360] Trial 0 finished with value: 0.3890298034242232 and parameters: {'n_neighbors': 14, 'p': 1}. Best is trial 0 with value: 0.3890298034242232.
[I 2024-10-14 16:55:06,688] Trial 1 finished with value: 0.38268864933417884 and parameters: {'n_neighbors': 21, 'p': 1}. Best is trial 0 with value: 0.3890298034242232.
[I 2024-10-14 16:55:23,536] Trial 2 finished with value: 0.5468188543648277 and parameters: {'n_neighbors': 18, 'p': 2}. Best is trial 2 with value: 0.5468188543648277.
[I 2024-10-14 16:55:31,343] Trial 3 finished with value: 0.38776157260621436 and parameters: {'n_neighbors': 15, 'p': 1}. Best is trial 2 with value: 0.5468188543648277.
[I 2024-10-14 16:55:39,482] Trial 4 finished with value: 0.3760304375396322 and parameters: {'n_neighbors': 26, 'p': 1}. Best is trial 2 with value: 0.5468188543648277.
[I 2024-10-14 16:55:47,306] Trial 5 fin